# DNA assembly project: ____

This notebook template shows a standard workflow of DNA assembly design, using EGF software. Each section begins with parameters that need to be defined.

## Obtain parts

In [ ]:
# List of all and new parts in assembly plan:
f1 = open('all_parts.txt', 'r')
f2 = open('new_parts.txt', 'r')

In [ ]:
all_parts = f1.read().splitlines()
new_parts = f2.read().splitlines()
f1.close()
f2.close()

In [ ]:
difference = set(new_parts) - set(all_parts)
if len(difference) != 0:
    print("Some parts are not in the plan:")
    print(difference)
else:
    print("All new parts are accounted for in the assembly. Retrieving these remaining parts from repository:")
    print(set(all_parts) - set(new_parts))

In [ ]:
# Alternatively retrieve from repository via API

---
## Evaluate manufacturability

In [8]:
import os
# Directory containing genbanks:
dir_to_evaluate = ""
results_target_dir = ""
pdf_evaluation = os.path.join(results_target_dir, "manufacturability_report.pdf")
excel_evaluation = os.path.join(results_target_dir, "manufacturability_report.xlsx")

In [8]:
try:
    os.mkdir(results_target_dir)
except:
    pass

In [4]:
import dnacauldron
import dnachisel
import dnachisel.reports.constraints_reports as cr

In [5]:
records_to_evaluate = dnacauldron.biotools.load_records_from_files(folder=dir_to_evaluate, use_file_names_as_ids=True)

In [6]:
constraints = [
    dnachisel.AvoidPattern("BsaI_site"),
    dnachisel.AvoidPattern("BsmBI_site"),
    dnachisel.AvoidPattern("BbsI_site"),
    dnachisel.AvoidPattern("SapI_site"),
    dnachisel.AvoidPattern("8x1mer"),
    dnachisel.AvoidPattern("5x3mer"),
    dnachisel.AvoidPattern("9x2mer"),
    dnachisel.AvoidHairpins(stem_size=20, hairpin_window=200),
    dnachisel.EnforceGCContent(mini=0.3, maxi=0.7, window=100),
    dnachisel.EnforceGCContent(mini=0.1, maxi=0.9, window=100),
    dnachisel.UniquifyAllKmers(k=15),
]

In [ ]:
dataframe = cr.constraints_breaches_dataframe(constraints, records_to_evaluate)
dataframe.to_excel(excel_evaluation)
records_annotated = cr.records_from_breaches_dataframe(dataframe, records_to_evaluate)
cr.breaches_records_to_pdf(records_annotated, pdf_evaluation)

## Sculpt sequences

In [5]:
import os
import dnachisel
import dnacauldron
import proglog
from Bio import SeqIO

In [3]:
dir_to_sculpt = ""

In [7]:
records_to_sculpt = dnacauldron.biotools.load_records_from_files(folder=dir_to_sculpt, use_file_names_as_ids=True)

In [ ]:
# for record in records_to_sculpt:
#     if not len(record) % 3 == 0:
#         print(record.id)

In [15]:
all_constraints=[
        dnachisel.AvoidPattern("BsaI_site"),
        dnachisel.AvoidPattern("NotI_site"),
        dnachisel.AvoidPattern("XbaI_site"),
        dnachisel.AvoidPattern("ClaI_site"),
        dnachisel.AvoidPattern("8x1mer"),
        dnachisel.builtin_specifications.UniquifyAllKmers(15),
        dnachisel.EnforceGCContent(mini=0.1, maxi=0.9, window=50),
#         dnachisel.EnforceTranslation(),
]

In [27]:
target_dir = ""
# For the ones that pass all constraints (edit loop if you have objectives):
target_dir_unsculpted = ""

In [18]:
try:
    os.mkdir(target_dir)
except:
    pass
try:
    os.mkdir(target_dir_unsculpted)
except:
    pass

In [ ]:
counter = 0
not_divisible_seqs = []
for record in records_to_sculpt:
    problem = dnachisel.DnaOptimizationProblem.from_record(record, extra_constraints=all_constraints)
    if not problem.all_constraints_pass():
        counter += 1
        print(record.id)
        len_seq = len(problem.sequence)
        print("Length:", len_seq)
        target_path = os.path.join(target_dir, record.id + ".zip")
        problem.optimize_with_report(target=target_path)
        print()
        print(problem.constraints_text_summary())
        print()
    else:  # just save the genbank
        genbank_filename = record.id + ".gb"
        with open(os.path.join(target_dir_unsculpted, genbank_filename), "w") as output_handle:
            SeqIO.write(record, output_handle, "genbank")
print()
print("Counter:", counter)

### Copy edited sequence files from reports:

Ensure reports are unzipped.

In [24]:
reports = os.listdir(target_dir)

In [25]:
from shutil import copyfile

In [ ]:
for report in reports:
    if report.endswith(".zip"):  # all should be lowercase
        continue

    gb_file = os.path.join(target_dir, report, "final_sequence.gb")
    
    target_filename = report + ".gb"
    target_file = os.path.join(target_dir_unsculpted, target_filename)
    print(gb_file, target_file)
    copyfile(gb_file, target_file)

---
## Domestication of new parts

In [ ]:
# Directory containing genbanks
dir_to_domesticate = ""
# Path to CSV of GoldenGateDomesticator spreadsheet
GGdomesticator_spreadsheet = ""
# Output path
domestication_target = ""

In [ ]:
import os
import genedom
import dnacauldron
# import proglog
# proglog.notebook()

In [ ]:
records_to_domesticate = dnacauldron.biotools.load_records_from_files(folder=dir_to_domesticate, use_file_names_as_ids=True)
EMMA_PLUS = genedom.GoldenGateDomesticator.standard_from_spreadsheet(GGdomesticator_spreadsheet)
genedom.batch_domestication(
    records=records_to_domesticate, 
    standard=EMMA_PLUS, 
    target=domestication_target)

In [ ]:
# Check if any names were truncated:
import pandas
order_ids = pandas.read_csv(os.path.join(domestication_target, "order_ids.csv"))

In [ ]:
any_truncated = False
for index, row in order_ids.iterrows():
    if row["sequence"] != row["order_id"]:
        any_truncated = True
        print("Truncated name:", end=" ")
        print(" --> ".join(row))
if not any_truncated:
    print("Part names were not truncated")

---

## Check overhangs

In [ ]:
import os
import overhang as oh

In [ ]:
projectname = "Project_name"
report_dir = ""
overhangs = ["TAGG", "ACGA"]
enzyme = "Esp3I"
kappagate_dataset = "2020_01h_Esp3I"  # or 2020_01h_BsaI

In [ ]:
overhangset = oh.OverhangSet(overhangs=overhangs, name=projectname, enzyme=enzyme)
oh.write_overhangset_report(os.path.join(report_dir, "overhang_report_" + projectname + ".pdf"), overhangset)
# Tatapov plot (37 Celsius, 1 hour):

In [ ]:
from kappagate import overhangs_list_to_slots, plot_circular_interactions, predict_assembly_accuracy, plot_colony_picking_graph, success_rate_facts

In [ ]:
slots = overhangs_list_to_slots(overhangs)
ax = plot_circular_interactions(
    slots, annealing_data=('37C', kappagate_dataset), rate_limit=200)
ax.figure.savefig(os.path.join(report_dir, "interactions_" + projectname + ".png"), bbox_inches='tight')

In [ ]:
predicted_rate, _, _ = predict_assembly_accuracy(slots)
ax = plot_colony_picking_graph(success_rate=predicted_rate)
ax.figure.savefig(os.path.join(report_dir, "success_rate_facts" + projectname + ".png"), bbox_inches='tight')

print(success_rate_facts(predicted_rate, plain_text=True))

---
## Cloning simulation

In [ ]:
# Dir of domesticated sequences
dir_domesticated = os.path.join(domestication_target, "domesticated_genbanks")
# Dir of available parts
dir_available_parts = ""
# Assembly plan folder prefix:
assembly_plan_name = "Assembly_plan"
assembly_plan_path = "assembly_plan.csv"
########################################
simulation_target_path = "predicted_simulation"

backbone_first = True
backbone_name = "HC_Amp_ccdB"

In [ ]:
import dnacauldron
repository = dnacauldron.SequenceRepository()
repository.import_records(folder=dir_domesticated, use_file_names_as_ids=True, topology="circular")
repository.import_records(folder=dir_available_parts, use_file_names_as_ids=True, topology="circular")

In [ ]:
repository.get_record(backbone_name).is_backbone = True

In [ ]:
assembly_plan = dnacauldron.AssemblyPlan.from_spreadsheet(
    name=assembly_plan_name,
    path=assembly_plan_path,
    assembly_class=dnacauldron.Type2sRestrictionAssembly
)

In [ ]:
simulation = assembly_plan.simulate(sequence_repository=repository)
stats = simulation.compute_stats()
print(stats)

In [ ]:
report_writer = dnacauldron.AssemblyReportWriter(
    include_assembly_plots=True,
    include_mix_graphs=True,
    include_pdf_report=True
)
simulation.write_report(simulation_target_path, assembly_report_writer=report_writer)

---

## Compare two sets of sequences (e.g. simulations) with GeneBlocks

In [ ]:
path_seq_1 = ""  # path to dir of sequence files
path_seq_2 = ""  # path to dir of sequence files
plot_export_path = "n1_compare_first_and_second_simulation"

---

In [ ]:
import os
try:
    os.mkdir(plot_export_path)
except:
    pass

In [ ]:
from Bio import SeqIO
seq_batch_1 = os.listdir(path_seq_1)
seq_batch_2 = os.listdir(path_seq_2)

In [ ]:
# The sequence names are assumed to be the same. Alternatively, a lookup-list dict can be implemented.

In [ ]:
seq_batch_1_records = {}
for seq in seq_batch_1:
    record = SeqIO.read(os.path.join(path_seq_1, seq), "genbank")
    # record.name = seq.split(".")[0]  # remove extension
    seq_batch_1_records[record.name] = record

In [ ]:
seq_batch_2_records = {}
for seq in seq_batch_2:
    record = SeqIO.read(os.path.join(path_seq_2, seq), "genbank")
    seq_batch_2_records[record.name] = record

In [ ]:
for sim, record in seq_batch_2_records.items():
    if str(seq_batch_1_records[sim].seq) != str(record.seq):
        print("Error:", sim)

In [ ]:
# if there is no output, then it's exactly the same

In [ ]:
import matplotlib.pyplot as plt
from geneblocks import DiffBlocks

In [ ]:
for sim, record in seq_batch_2_records.items():
    compare_record = seq_batch_1_records[sim]
    
    diff_blocks = DiffBlocks.from_sequences(record, compare_record)
    ax1, ax2 = diff_blocks.merged().plot(figure_width=8)
    filename = record.name + ".png"
    ax1.figure.savefig(os.path.join(plot_export_path, filename))

---
## Calculate total length of DNA domesticated (bp)

In [ ]:
parts_to_order = dnacauldron.biotools.load_records_from_files(folder=dir_domesticated, use_file_names_as_ids=True)

In [ ]:
total_length = 0
for part in parts_to_order:
    total_length += len(part.seq)
print(total_length)

In [ ]:
# Check parts that are too short
length_cutoff = 100
for part in parts_to_order:
    if len(part) < length_cutoff:
        print("Too short:", part.id)

---
## Export as a table

Required by some DNA synthesis companies

In [ ]:
import pandas

name = []
seq = []
for seqrecord in parts_to_order:
    name += [seqrecord.id]
    seq += [str(seqrecord.seq)]

d = {'name': name,
    'seq': seq}
df = pandas.DataFrame(d)

df.to_csv("projectname.csv", index=False)